In [ ]:
# !pip install torch==1.8.1+cu102 torchvision==0.9.1+cu102 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html



In [ ]:
# !pip install easyocr

In [ ]:
# !pip install pytesseract

In [ ]:
# !pip install pyautogui

In [ ]:
# import easyocr
import cv2
import matplotlib.pyplot as plt
import numpy as np
import re
import os

In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
# # If EasyOCR gives error #15
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
fileLoc = r'C:\Users\ZF\Desktop\AA _RPA_Challenge\Challange4\Invoices'

In [ ]:
custom_config = r'--oem 3 --psm 6'

In [ ]:
invoices_lst = [i for i in os.listdir(fileLoc) if i.endswith('.tiff')]

In [ ]:
fileList = [fileLoc+"\\"+i for i in invoices_lst]

In [ ]:
invoice = {}

for f in fileList:
    
    invoice[f] = {}
    
    img = cv2.imread(f)

    # OCR - get text
    # str_list = pytesseract.image_to_string(img, config=custom_config).split('\n')
    str_list = pytesseract.image_to_string(img).split('\n')
    str_list2 = [i for i in str_list if re.match('[\d\w]+?', i)]

    # Extract Data
    qty_idx = 0
    subtotal_idx = 0

    for idx, i in enumerate(str_list2):

        if "Invoice" in i:

            i_list = i.split(' ')

            for jdx, j in enumerate(i_list):

                if j == 'no.':
                    try:
                        invoice_num = i_list[jdx+1]
                    except:
                        invoice_num = str_list2[idx+1].split(' ')[-1]

                if j == 'Date':
                    try:
                        invoice_date = ' '.join([i_list[jdx+1], i_list[jdx+2], i_list[jdx+3]])
                    except:
                        invoice_date = ' '.join(str_list2[idx+1].split(' ')[-3:])


                if j == 'Amount':
                    try:
                        invoice_amount = i_list[-1]
                    except:
                        invoice_amount = str_list2[idx+1].split(' ')[-1]
                        
                    invoice_amount = invoice_amount.replace('$', '').replace(',', '')

        if "Quantity" in i:
            qty_idx = idx


        if "Subtotal" in i:
            subtotal_idx = idx
        
    # Collate results
    
    invoice[f]['invoice_num'] = invoice_num
    invoice[f]['invoice_date'] = invoice_date
    invoice[f]['invoice_amt'] = invoice_amount
    invoice[f]['items'] = []
    
    # Collate invoice items
    for i in range(qty_idx+1, subtotal_idx):
        i_split = str_list2[i].replace('|', '').split(' ')
        i_split = [i for i in i_split if i != '']

        g_index = i_split.index('G')

        qty = i_split[0]
        total_amt = i_split[-1].replace('$', '').replace(',', '')
        desc = ' '.join(i_split[2:g_index-1])
        
        invoice[f]['items'].append([qty, desc, total_amt])

In [ ]:
invoice

In [ ]:
#rpa proper
import rpa as r
import pyautogui

challenge_site = r'https://developer.automationanywhere.com/challenges/automationanywherelabs-invoiceentry.html'

r.tagui_location()


In [ ]:
r.init(visual_automation = True, turbo_mode=True)

In [ ]:
r.url(challenge_site)
r.wait(2)

In [ ]:
# pyautogui.click('aa.png')

In [ ]:
invoice_counter = 0

for fileName, invoiceInfo in invoice.items():
    r.type("//input[@id='invoiceNumber']", invoiceInfo['invoice_num'])
    r.type("//input[@id='invoiceDate']", invoiceInfo['invoice_date'])
    r.type("//input[@id='invoiceTotal']", invoiceInfo['invoice_amt'])
    
    num_items = len(invoiceInfo['items'])
    
    for _ in range(num_items-1):
        r.click("//button[contains(text(),'Add')]")
    
    for idx, items in enumerate(invoiceInfo['items']):
        qty, desc, amt = items
        r.type(f"//input[@id='quantity_row_{idx+1}']", qty)
        r.type(f"//input[@id='description_row_{idx+1}']", desc)
        r.type(f"//input[@id='price_row_{idx+1}']",amt)
        
    # Upload file    
    r.upload("#fileupload", fileName)
#     r.wait(1.5)
# #     fw = pyautogui.getWindowsWithTitle('Open')
# #     fw=fw[0]
# #     pyautogui.click(fw.center)
#     pyautogui.write(str(fileName))
#     pyautogui.write(['enter'])
    
    # Acknowledge
    r.click("//input[@id='agreeToTermsYes']")
    
    r.click("//button[normalize-space()='Submit']")
    
    print(fileName)
    invoice_counter += 1
    
    

In [ ]:
# r.close()

In [ ]:
# r.upload("#fileupload", list(invoice.keys())[0])